In [1]:
!pip install tensorflow
!pip install transformers
!pip install torch

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import Dense, Input,BatchNormalization,Dropout,Embedding,Flatten,SimpleRNN
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
data = pd.read_csv('our_data.csv', delimiter = ';', lineterminator = '\r')

In [4]:
# dropping last row
data = data[:-1]

In [5]:
data

,text,example_very_unclear,joy,anger,desire,curiosity,disappointment,confusion,label
0,\nI am super happy for you!!,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,\nOh very very good,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,\nThank youuu!!!,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,\nHahahahahah,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,\nLook I hesitated between Maksime and Maxime ...,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
591,\nNeither do i,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0
592,\nIn person,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,\nA friend of mine told that we should have en...,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0
594,\nOsnakjlfafv,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# creating mapping of emotion to number
dct = {0 : 'joy', 1 : 'anger', 2 : 'desire', 3:'curiosity', 4 : 'disappointment', 5 : 'confusion'}

In [13]:
# fetching only raws that is clear
data = data[data['example_very_unclear'] == False]

In [14]:
data
data['label'] = data['label'] - 1
data.head()

/tmp/ipykernel_3423825/2881916659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data['label'] - 1


,text,example_very_unclear,joy,anger,desire,curiosity,disappointment,confusion,label
0,\nI am super happy for you!!,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,\nOh very very good,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,\nThank youuu!!!,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,\nHahahahahah,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,\nLook I hesitated between Maksime and Maxime ...,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
for _ in range(50):
  data = data.sample(frac=1)
  

In [31]:
data.label.unique()

array([5, 2, 3, 0, 1, 4])

# **Using Bert Algorithm**

# converting one hot vector of emotion to number

In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io
# % matplotlib inline

In [20]:
# identify and specify the GPU as the device, later in training loop we will load data into device
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

In [21]:
#device = torch.device("cuda")

In [22]:
data.drop(['example_very_unclear'], axis = 1, inplace = True)

In [23]:
data

,text,joy,anger,desire,curiosity,disappointment,confusion,label
554,\nis it just me or MAA105 final exam is 3h long?,0.0,0.0,0.0,0.0,0.0,1.0,5.0
188,"\n""Don't hesitate to send me pictures :)\n""",0.0,0.0,1.0,0.0,0.0,0.0,2.0
327,"\n""are u here on Friday?\n""",0.0,0.0,0.0,1.0,0.0,0.0,3.0
238,"\n""For you want to go to the shop together, ma...",0.0,0.0,1.0,0.0,0.0,0.0,2.0
483,"\n""Oh I see... it's a bit confusing though\n""",0.0,0.0,0.0,0.0,0.0,1.0,5.0
...,...,...,...,...,...,...,...,...
116,"\ncalm down, youre not the only one cleaning t...",0.0,1.0,0.0,0.0,0.0,0.0,1.0
511,"\n""Where u at?\n""",0.0,0.0,0.0,0.0,0.0,1.0,5.0
198,"\n""Ahh I like to watch you play That's so nic...",0.0,0.0,1.0,0.0,0.0,0.0,2.0
455,"\n""both options are bad\n""",0.0,0.0,0.0,0.0,1.0,0.0,4.0


In [24]:
## create label and sentence list
sentences = data.text.values

#check distribution of data based on labels
print("Distribution of data based on labels: ",data.label.value_counts())

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room for more anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 256

## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]
labels = data.label.values

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

Distribution of data based on labels:  2.0    113
0.0    103
3.0    101
4.0     80
1.0     79
5.0     78
Name: label, dtype: int64


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Actual sentence before tokenization:  
"are u here on Friday?
"
Encoded Input from dataset:  [101, 1000, 2024, 1057, 2182, 2006, 5958, 1029, 1000, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

/home/mert/miniconda3/envs/olymposc/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,labels,random_state=41,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=41,test_size=0.1)

In [26]:
data = data.dropna()
data['label'] = [int(x) for x in data['label']]

/tmp/ipykernel_3423825/2787345946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = [int(x) for x in data['label']]


In [34]:
data.label.isna().unique()

array([False])

In [28]:
# convert all our data into torch tensors, required data type for my model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 192

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [29]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6).to(device)

# Parameters:
lr = 2e-5
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [32]:
## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for _ in tnrange(1,epochs+1,desc='Epoch'):
  print( F" Epoch {_} ")
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    print(b_labels)
    b_labels.dropna
    b_input_ids, b_input_mask, b_labels = batch
    b_labels = F.one_hot(b_labels.to(torch.int64), num_classes=6)

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    
    # Backward pass
    loss.backward()
    
    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()
    
    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    
    df_metrics=pd.DataFrame({'Epoch':epochs,'Actual_class':labels_flat,'Predicted_class':pred_flat})
    
    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)
    
    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')

/tmp/ipykernel_3423825/878521272.py:9: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for _ in tnrange(1,epochs+1,desc='Epoch'):


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

 Epoch 1 
tensor([1., 0., 4., 4., 1., 3., 0., 3., 4., 0., 1., 2., 4., 3., 2., 1., 5., 5.,
        2., 0., 4., 2., 2., 4., 2., 3., 3., 2., 0., 1., 1., 0., 4., 0., 1., 5.,
        5., 2., 2., 5., 2., 3., 4., 3., 0., 0., 1., 4., 2., 2., 0., 0., 4., 4.,
        4., 5., 5., 3., 2., 3., 5., 1., 1., 0., 1., 3., 1., 3., 1., 1., 2., 0.,
        1., 1., 0., 4., 3., 2., 3., 3., 4., 2., 3., 0., 0., 4., 1., 4., 3., 0.,
        2., 2., 0., 2., 3., 0., 1., 0., 4., 0., 2., 4., 0., 3., 1., 2., 2., 5.,
        1., 5., 5., 3., 3., 3., 1., 1., 5., 4., 4., 5., 3., 2., 4., 5., 3., 2.,
        3., 1., 5., 0., 0., 5., 5., 2., 2., 0., 5., 5., 0., 0., 0., 0., 1., 0.,
        5., 3., 2., 0., 1., 2., 0., 3., 4., 0., 4., 3., 1., 4., 1., 5., 3., 2.,
        0., 4., 2., 1., 0., 4., nan, 1., 0., 3., 3., 3., 5., 2., 3., 5., 2., 0.,
        4., 3., 0., 1., 2., 5., 3., 0., 1., 2., 2., 5.], dtype=torch.float64)


RuntimeError: Class values must be non-negative.